In [1]:
from dash import Dash, dcc, html, dcc, callback, Output, Input
import dash_bootstrap_components as dbc
import plotly.express as px
import pandas as pd
import numpy as np
import re
from functools import reduce
app = Dash(__name__)
# styles
colors = {
    'background': '#d9ffb3',
    'text': '#0066cc',
    'title' : '#000066'
}

In [2]:
df_growth = pd.read_excel('..\Data\LinkedIn_DigitalData\public_use-industry-employment-growth.xlsx', sheet_name=['Growth from Industry Transition'])['Growth from Industry Transition']
df_growth = df_growth.rename(columns={'growth_rate_2015': '2015', 'growth_rate_2016': '2016', 'growth_rate_2017': '2017', 'growth_rate_2018': '2018', 'growth_rate_2019': '2019'})
# df_growth

In [3]:
@app.callback(
    Output("growth-line", "figure"),
    [Input("country-filter-2", "value"),
    Input("sector-filter-2", "value")]
)
def update_output(country, section):
    filtered_df = df_growth[df_growth['country_name']==country]
    filtered_df = filtered_df[filtered_df['isic_section_name']==section]
    melted = filtered_df.melt(id_vars=['industry_name'], value_vars=['2015','2016','2017','2018','2019'], var_name='year', value_name='growth_rate')
    fig = px.line(
            melted,
            x="year",
            y="growth_rate",
            color="industry_name",
            markers=True,
            title="Growth rate over the years",
        )
    fig.add_hline(y=0, line=dict(color='red',dash='dash'))
    for trace in fig.data:
        trace.update(visible='legendonly')
    fig.update_layout(plot_bgcolor=colors['background'], paper_bgcolor=colors['background'])
    return fig

In [4]:
app.layout = html.Div(style={'backgroundColor': colors['background'], 'margin': 10, 'padding': 20}, children=[
    html.H1(
        children='Industry Growth Rates',
        style={
            'textAlign': 'center',
            'color': colors['title']
        }
    ),

     dbc.Row([
                html.Label("Country", style={'color': colors['text']}),
                dcc.Dropdown(
                    id="country-filter-2",
                    options=[{"label": s, "value": s}
                             for s in df_growth['country_name'].dropna().unique()],
                    value='India',
                    style={
                        "width": "75%",  # set the width of the dropdown to 50% of its container
                        "height": "50%",  # set the height of each dropdown item to 40 pixels
                        "backgroundColor": colors['background'],  # set the background color of the dropdown to dark gray
                        "color": "#000",  # set the font color of the dropdown to light gray
                        "fontFamily": "sans-serif"  # set the font family of the dropdown to sans-serif
                    }
                ),
                html.Label("Sector", style={'color': colors['text']}),
                dcc.Dropdown(
                    id="sector-filter-2",
                    options=[{"label": s, "value": s}
                             for s in df_growth['isic_section_name'].dropna().unique()],
                    value='Information and communication',
                    style={
                        "width": "75%",  # set the width of the dropdown to 50% of its container
                        "height": "50%",  # set the height of each dropdown item to 40 pixels
                        "backgroundColor": colors['background'],  # set the background color of the dropdown to dark gray
                        "color": "#000",  # set the font color of the dropdown to light gray
                        "fontFamily": "sans-serif"  # set the font family of the dropdown to sans-serif
                    }
                ),
         dcc.Graph(id="growth-line")
             
     ])
])

In [5]:
if __name__ == "__main__":
    app.run_server(debug=True)